In [1]:
from math import sqrt
from skimage.color import rgb2gray
from skimage import io
from skimage import filters
from skimage.util import crop
from skimage.util import invert
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import os.path
import skimage as skimage
from skimage import morphology
from scipy import ndimage

In [2]:
surfaceDistance = 1 ## here you can control how far the slice should be distant to the surface of the mask
safetyBorder = 3    ## the number of bottom and top slices that will be set to zero (to prevent mask border touching the image border)
padding = 1       ## can be used to extract thicker peels

In [8]:
def SurfaceExtraction(maskImage,rawImage):    
    ##set border part to zero to prevent masks touching the border
    maskImage[:, :, 1:safetyBorder] = 0
    maskImage[:, :, -1:-(safetyBorder+1)] = 0
    maskImage = morphology.closing(maskImage)
    maskImage = ndimage.distance_transform_edt(maskImage)
    maskImage = ndimage.gaussian_filter(maskImage, sigma =1)
    maskImage = (maskImage > (surfaceDistance-padding)) & (maskImage < (surfaceDistance+padding))

    ##flip the image to have the XZ cross section
    rotatedRawImage = ndimage.rotate(rawImage, 90, (0,2))
    rotatedMaskImage = ndimage.rotate(maskImage, 90, (0,2))

    ##label the surface regions and extract their volume
    labeledShellImage, number_of_objects = ndimage.label(rotatedMaskImage)
    regionProps = skimage.measure.regionprops(labeledShellImage)
    volumes = []
    pixelIdxLists = []
    for i in range (0,number_of_objects): 
        if regionProps[i].area > 5000:## minimum volume bounded by a surface
            volumes = np.append(volumes,regionProps[i].area)
            pixelIdxLists = np.append(pixelIdxLists,regionProps[i].coords)
            print (i, regionProps[i].area)
    a = len(pixelIdxLists)/3       
    pixelIdxLists = pixelIdxLists.reshape((int(a),3))
    
    ##identify the inner and outer surface based on volume
    SurfaceOne = np.zeros(rotatedMaskImage.shape)
    for i in range (0,int(volumes[0]+1)):
        ao,bo,co = pixelIdxLists[i]
        SurfaceOne[int(ao),int(bo),int(co)] = rotatedRawImage[int(ao),int(bo),int(co)]

    SurfaceTwo = np.zeros(rotatedMaskImage.shape)
    for i in range (int(volumes[0]+1),int(volumes[1])+1):
        ai,bi,ci = pixelIdxLists[i]
        SurfaceTwo[int(ai),int(bi),int(ci)] = rotatedRawImage[int(ai),int(bi),int(ci)]
        
    if volumes[0]>volumes[1]:
        outerSurface = SurfaceOne
        innerSurface = SurfaceTwo
    if volumes[0]<volumes[1]:
        outerSurface = SurfaceTwo
        innerSurface = SurfaceOne        
    
    return outerSurface,innerSurface

In [62]:
def ind2sub(array_shape, ind):
    ind[ind < 0] = -1
    ind[ind >= array_shape[0]*array_shape[1]] = -1
    rows = (ind.astype('int') / array_shape[1])
    cols = ind % array_shape[1]
    return (rows, cols)

def SurfaceProjection(rawImage):
    ##get the image size
    imageSize = rawImage.shape
    
    ##initialize the result image and the start location
    resultImage = np.zeros([imageSize[2], 3000]);    
    startLocation = [];
    
    ##extract the valid peel pixels
    for i in  range (1,imageSize[2]):
    ##get the current slice and skeletonize the peel to have only one pixel thick boundaries
    ##use the peel to mask the raw image
        currentSlice = np.squeeze(rawImage[:,:,i])         
        currentSliceBinary = morphology.medial_axis((currentSlice > 0))
        currentSlice = currentSlice * currentSliceBinary
        a,b = currentSlice.shape
        
    ##initialize the visited pixels array and extract the valid pixels
        visitedPixels = np.zeros([int(a),int(b)])
        validPixels = currentSlice > 0
        [xpos, ypos] = ind2sub([int(a),int(b)], validPixels)
        positions = np.sort(np.reshape([xpos, ypos],[-2,1]))
        
    ##set the start location as the lowest, right-most mask pixel
    ##start positions at subsequent slices are based on a nearest neighbor search to the previous location
        if (len(startLocation)>0):
            startLocation = positions[0,:]
        else:
            distances = sqrt((positions[0,:] - startLocation[0])**2 + (positions[:,1] - startLocation[1])**2)
            [minDist, minIndex] = min(distances)
            startLocation = positions[minIndex,:]

        previousNeighbor = 1
    
    return startLocation


In [104]:
int(positions[0,:])

0

In [86]:
startLocation=[2,4]

In [99]:
(int(positions[:,0]) - int(startLocation[1])**2)

TypeError: only size-1 arrays can be converted to Python scalars

In [94]:
distances = sqrt((int(positions[0,:]) - int(startLocation[0]))**2 + (int(positions[:,1]) - int(startLocation[1])**2))
[minDist, minIndex] = min(distances)
startLocation = positions[minIndex,:]

IndexError: index 1 is out of bounds for axis 1 with size 1

In [93]:
distances = sqrt((int(positions[0,:]) - int(startLocation[0]))**2 + (int(positions[:,1]) - int(startLocation[1])**2)


SyntaxError: invalid syntax (<ipython-input-93-67150b4cb2a2>, line 2)

In [76]:
int(positions[0,:]) - int(startLocation[0])

IndexError: list index out of range

In [63]:
SurfaceProjection(rawImage)

IndexError: list index out of range

In [4]:
## open the current raw image file
rawImage = io.imread('Data/GAP43mCardinal_Embryo1_T0.tif', plugin='pil')
rawImage = rawImage[:,200:230,:] ## crop if needed
##set border part to zero to prevent masks touching the border.
maskImage = io.imread('Data/Masked/GAP43mCardinal_Embryo1_T0_Masked.tif', plugin='pil') > 0
maskImage = maskImage[:,200:230,:] ## crop if needed

In [9]:
outerSurface,innerSurface = SurfaceExtraction(maskImage,rawImage)

0 258885
148 272862
1227 82033


##get the image size
imageSize = rawImage.shape

            %% initialize the result image and the start location
            resultImage = zeros(imageSize(3), 3000);    
            startLocation = [];

            %% extract the valid peel pixels
            for i=1:imageSize(3)

                %% get the current slice and skeletonize the peel to have only one pixel thick boundaries
                %% use the peel to mask the raw image
                currentSlice = squeeze(rawImage(:,:,i));            
                currentSliceBinary = bwmorph(currentSlice > 0, 'skel', Inf);
                currentSlice = double(currentSlice) .* double(currentSliceBinary);
                sliceSize = size(currentSlice);
                
                %% initialize the visited pixels array and extract the valid pixels
                visitedPixels = zeros(sliceSize);
                validPixels = find(currentSlice > 0);
                [xpos, ypos] = ind2sub(sliceSize, validPixels);
                positions = sortrows([xpos, ypos], [-2,1]);

                %% set the start location as the lowest, right-most mask pixel
                %% start positions at subsequent slices are based on a nearest neighbor search to the previous location
                if (isempty(startLocation))
                    startLocation = positions(1,:);
                else
                    distances = sqrt((positions(:,1) - startLocation(1)).^2 + (positions(:,2) - startLocation(2)).^2);
                    [minDist, minIndex] = min(distances);
                    startLocation = positions(minIndex,:);
                end
                previousNeighbor = 1;

                %% scan the peel using only a single consistent scan direction for all peels.
                positionQueue = [startLocation, 1];
                while ~isempty(positionQueue)
                    
                    %% get the current position from the queue and remove the top entry
                    currentPosition = positionQueue(1,:);
                    positionQueue(1,:) = [];

                    %% set the intensity value of the current peel position to the results image
                    resultImage(i, floor(currentPosition(3))) = currentSlice(currentPosition(1), currentPosition(2));
                    resultImage(i, ceil(currentPosition(3))) = currentSlice(currentPosition(1), currentPosition(2));
                   
                    %% plot progress pixel by pixel if debug figres are enabled
                    if (debugFigures == true)
                        figure(1); clf; hold on;
                        imagesc(currentSlice);
                        plot(currentPosition(2), currentPosition(1), '*r');
                    end

                    %% set current position to visited
                    if (visitedPixels(currentPosition(1), currentPosition(2)) > 0)
                        continue;
                    end
                    
                    %% perform boundary tracing using Moore's algorithm (CCW)
                    nextIndices = [];
                    for n=0:7
                        
                        %% compute the potential next neighbor in CCW fashion
                        potentialNeighbor = mod(previousNeighbor-1 + n, 8) + 1;
                        
                        %% get the displacements of the neighbors from the LUT
                        j = neighborList(potentialNeighbor, 1);
                        k = neighborList(potentialNeighbor, 2);
                        
                        % prevent out of bounds errors
                        if ((currentPosition(1)+j) < 1 || ...
                            (currentPosition(1)+j) > sliceSize(1) || ...
                            (currentPosition(2)+k) < 1 || ...
                            (currentPosition(2)+k) > sliceSize(2) || ...
                            (j == 0 && k == 0))
                            continue;
                        end
                        
                        %% plot candidate search if enabled
                        if (debugFigures == true)
                            plot(currentPosition(2)+k, currentPosition(1)+j, '.r');
                        end

                        %% continue if the candidate was already visited before
                        if (visitedPixels(currentPosition(1)+j, currentPosition(2)+k) > 0)
                            continue;
                        end

                        %% select the minimum distance candidate in the correct direction
                        if (currentSlice(currentPosition(1)+j, currentPosition(2)+k) > 0)
                            nextIndices = [j, k];
                            previousNeighbor = find(neighborList(:,1) == -j & neighborList(:,2) == -k);
                            
                            if (debugFigures == true)
                                plot(currentPosition(2)+k, currentPosition(1)+j, 'og');
                            end
                            break;
                        end 
                    end

                    %% add the identified closest neighbor to the queue and reset the last direction
                    if (~isempty(nextIndices))
                        positionQueue = unique([positionQueue; currentPosition(1)+nextIndices(1), currentPosition(2)+nextIndices(2), currentPosition(3) + sqrt(nextIndices(1)^2 + nextIndices(2)^2)], 'rows');
                        previousNeighbor = 1;
                    end

                    %% set current position to visited
                    visitedPixels(currentPosition(1), currentPosition(2)) = 1;

                    if (debugFigures == true)
                        drawnow;
                    end
                end

                %% status
                disp(['Finished processing ' num2str(i) ' / ' num2str(imageSize(3)) ' slices ...']);
            end
